In [11]:
import pandas as pd
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [9]:
# json_file = Path()
df = pd.DataFrame(columns=['date', 'chamber', 'speaker', 'speaker_id','text'])
json_file =  Path.cwd().parent.joinpath(r'output\2020\CREC-2020-01-08\json\CREC-2020-01-08-pt1-PgE11.json')
with open(json_file) as f:
    data = json.load(f)
    spech_date = data['header']['month'] + ' ' + data['header']['day'] + ', ' + data['header']['year']
    chamber = data['header']['chamber']
    for speech in data['content']:
        if speech['kind'] == 'speech':
            speaker = speech['speaker']
            speaker_id = speech['speaker_bioguide']
            text = speech['text']
            df = df.append({'date': spech_date, 'chamber': chamber, 
                            'speaker': speaker, 'speaker_id': speaker_id, 
                            'text': text}, ignore_index=True)
df

C:\Users\jesse\AppData\Local\Temp\ipykernel_30444\3475167099.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': spech_date, 'chamber': chamber,


,date,chamber,speaker,speaker_id,text
0,"January 8, 2020",House,Mr. GONZALEZ of Texas,G000581,"Mr. GONZALEZ of Texas. Madam Speaker, I was ..."


In [10]:
def scrape_speeches(json_file):
    """
    Scrape speech text from an individual json file
    """
    temp_df = pd.DataFrame(columns=['date', 'chamber', 'speaker', 'speaker_id','text'])
    with open(json_file) as f:
        data = json.load(f)
        spech_date = data['header']['month'] + ' ' + data['header']['day'] + ', ' + data['header']['year']
        chamber = data['header']['chamber']
        for speech in data['content']:
            if speech['kind'] == 'speech':
                speaker = speech['speaker']
                speaker_id = speech['speaker_bioguide']
                text = speech['text']
                temp_df = temp_df.append({'date': spech_date, 'chamber': chamber, 
                                'speaker': speaker, 'speaker_id': speaker_id, 
                                'text': text}, ignore_index=True)
    return temp_df

In [12]:
# iterate through all json files in the output folder
# and scrape the speech text
speech_df = pd.DataFrame(columns=['date', 'chamber', 'speaker', 'speaker_id','text'])
for json_file in Path.cwd().parent.joinpath(r'output').rglob('*.json'):
    temp_df = scrape_speeches(json_file)
    speech_df = speech_df.append(temp_df, ignore_index=True)
speech_df

,date,chamber,speaker,speaker_id,text
0,"January 2, 2020",House,The SPEAKER pro tempore,None,The SPEAKER pro tempore. Pursuant to section...
1,"January 2, 2020",House,The SPEAKER pro tempore,None,The SPEAKER pro tempore. The Chair will lead...
2,"January 2, 2020",House,The SPEAKER pro tempore,None,The SPEAKER pro tempore. Pursuant to section...
3,"January 2, 2020",Senate,The PRESIDING OFFICER,None,The PRESIDING OFFICER. The clerk will please...
4,"January 2, 2020",Senate,The ACTING PRESIDENT pro tempore,None,The ACTING PRESIDENT pro tempore. Under the ...
...,...,...,...,...,...
83649,"February 16, 2023",Senate,The PRESIDING OFFICER (Mr. Cardin),None,The PRESIDING OFFICER (Mr. Cardin). The Sena...
83650,"February 16, 2023",Senate,Mr. MURPHY,M001169,"Mr. MURPHY. Mr. President, I rise today to o..."
83651,"February 16, 2023",Senate,Mr. CARDIN,C000141,"Mr. CARDIN. Mr. President, I ask unanimous c..."
83652,"February 16, 2023",Senate,The PRESIDING OFFICER,None,"The PRESIDING OFFICER. Without objection, it..."


In [24]:
# filter out rows where the word 'None' is contained in the speaker_id
speech_df = speech_df[~speech_df['speaker_id'].fillna('None').str.contains('None')]
speech_df.reset_index(drop=True, inplace=True)
speech_df['text_len'] = speech_df['text'].str.len()
speech_df.sort_values(by='text_len', ascending=False, inplace=True)
speech_df

,date,chamber,speaker,speaker_id,text,text_len
2238,"January 23, 2020",Senate,Mr. McCONNELL,M000355,"Mr. McCONNELL. Mr. Chief Justice, if I may, ...",90902
23257,"October 25, 2020",Senate,Mr. MURPHY,M001169,"Mr. MURPHY. Madam President, first, let me t...",68193
2301,"January 24, 2020",Senate,Mr. McCONNELL,M000355,Mr. McCONNELL. I have spoken with Congressma...,62366
5924,"February 25, 2020",Senate,Mr. LANKFORD,L000575,"Mr. LANKFORD. Madam President, the country i...",60888
6456,"February 28, 2020",House,Mr. KING of Iowa,K000362,"Mr. KING of Iowa. Madam Speaker, it is my ho...",57357
...,...,...,...,...,...,...
54484,"October 21, 2021",House,Mr. ROY,R000614,Mr. ROY. Amen.,16
54450,"October 21, 2021",House,Mr. ROY,R000614,Mr. ROY. Yes.,15
54460,"October 21, 2021",House,Mr. ROY,R000614,Mr. ROY. Yes.,15
30099,"January 27, 2021",Senate,Mr. DURBIN,D000563,Mr. DURBIN.\n\n,15


In [25]:
speech_df.to_csv('congressional_record_speeches.csv.gzip', compression='gzip', index=False)